In [2]:
import string
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Function to extract the Nth most likely word while filtering out punctuation
def fetch_nth_token(model, tokenizer, prompt, n):
    # Tokenize input
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Get model predictions
    with torch.no_grad():
        output = model(input_ids)
        # Last word prediction
        logits = output.logits[0, -1]
        probs = torch.nn.functional.softmax(logits, dim=-1)

    # Filter out punctuation tokens
    token_ids = torch.arange(probs.size(0))
    non_punct_tokens = [tid.item() for tid in token_ids if tokenizer.decode([tid]).strip().isalpha()]

    if not non_punct_tokens:
        # Fallback if no valid tokens exist
        return None

    filtered_indices = torch.tensor(non_punct_tokens, device=probs.device)
    filtered_probs = probs[filtered_indices]

    # Fetch the nth most probable word, or fallback
    if len(filtered_probs) >= n:
        nth_token_id = filtered_indices[torch.topk(filtered_probs, k=n).indices[-1]].item()
    else:
        nth_token_id = filtered_indices[-1].item()

    return tokenizer.decode([nth_token_id]).strip()

# Function to preserve punctuation while replacing only the last word
def replace_last_word(line, new_word):
    words = line.split()
    if not words:
        # Return empty lines unchanged
        return line

    last_word = words[-1]

    # Check if the last word has punctuation
    stripped_word = last_word.rstrip(string.punctuation)
    # Get the punctuation
    trailing_punct = last_word[len(stripped_word):]

    # Replace last word while keeping punctuation
    return " ".join(words[:-1] + [new_word + trailing_punct])

# Load GPT-2 model and tokenizer
gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Original poem - The Snow Man by Wallace Stevens (1879-1955)
poem_lines = [
    "One must have a mind of winter",
    "To regard the frost and the boughs",
    "Of the pine-trees crusted with snow;",
    "And have been cold a long time",
    "To behold the junipers shagged with ice,",
    "The spruces rough in the distant glitter",
    "Of the January sun; and not to think",
    "Of any misery in the sound of the wind,",
    "In the sound of a few leaves,",
    "Which is the sound of the land",
    "Full of the same wind",
    "That is blowing in the same bare place",
    "For the listener, who listens in the snow,",
    "And, nothing himself, beholds",
    "Nothing that is not there and the nothing that is."
]

# Print the original poem
print("\n=== Original Poem ===")
for line in poem_lines:
    print(line)

# Apply P+7 transformation
modified_poem_p7 = []
modified_poem_px = []

for line in poem_lines:
    words = line.split()
    if not words:
        modified_poem_p7.append(line)
        modified_poem_px.append(line)
        continue

    last_word = words[-1].rstrip(string.punctuation)

    new_word_p7 = fetch_nth_token(gpt2, tokenizer, " ".join(words), 7) or last_word
    new_word_px = fetch_nth_token(gpt2, tokenizer, " ".join(words), 35) or last_word

    modified_poem_p7.append(replace_last_word(line, new_word_p7))
    modified_poem_px.append(replace_last_word(line, new_word_px))

# Print the modified poem (P+7)
print("\n=== Modified Poem (P+7) ===")
for line in modified_poem_p7:
    print(line)

# Print the modified poem (P+35)
print("\n=== Modified Poem (P+35) ===")
for line in modified_poem_px:
    print(line)



=== Original Poem ===
One must have a mind of winter
To regard the frost and the boughs
Of the pine-trees crusted with snow;
And have been cold a long time
To behold the junipers shagged with ice,
The spruces rough in the distant glitter
Of the January sun; and not to think
Of any misery in the sound of the wind,
In the sound of a few leaves,
Which is the sound of the land
Full of the same wind
That is blowing in the same bare place
For the listener, who listens in the snow,
And, nothing himself, beholds
Nothing that is not there and the nothing that is.

=== Modified Poem (P+7) ===
One must have a mind of or
To regard the frost and the which
Of the pine-trees crusted with but;
And have been cold a long so
To behold the junipers shagged with he,
The spruces rough in the distant in
Of the January sun; and not to about
Of any misery in the sound of the let,
In the sound of a few and,
Which is the sound of the on
Full of the same from
That is blowing in the same bare I
For the listener, 